In [1]:
!python libraries.py

2024-08-30 17:54:10,156 - INFO - Upgrading pip...
2024-08-30 17:54:10,156 - INFO - Running pip command: install --upgrade pip
2024-08-30 17:54:11,462 - INFO - Command 'install --upgrade pip' executed successfully.
2024-08-30 17:54:11,472 - INFO - pip upgraded successfully.
2024-08-30 17:54:11,472 - INFO - Installing library numpy...
2024-08-30 17:54:11,472 - INFO - Running pip command: install numpy
2024-08-30 17:54:12,720 - INFO - Command 'install numpy' executed successfully.
2024-08-30 17:54:12,720 - INFO - numpy installed successfully.
2024-08-30 17:54:12,720 - INFO - Installing library pandas...
2024-08-30 17:54:12,730 - INFO - Running pip command: install pandas
2024-08-30 17:54:13,968 - INFO - Command 'install pandas' executed successfully.
2024-08-30 17:54:13,968 - INFO - pandas installed successfully.
2024-08-30 17:54:13,968 - INFO - Installing library matplotlib...
2024-08-30 17:54:13,968 - INFO - Running pip command: install matplotlib
2024-08-30 17:54:15,227 - INFO - Comman

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7436468564582840846
xla_global_id: -1
]


In [4]:
import pickle

In [5]:
import os
from config import Config

config = Config(
    data_dir='./data',
    train_data_subdir='train/',
    val_data_subdir='test/',
    test_data_subdir='test/',
    models_dir='./models',
    dl_models_subdir='dl/',
    ml_models_subdir='ml/'
)

data_dir = ['.ipynb_checkpoints', 'features.txt', 'targets.txt', 'test', 'test_file_idx.txt', 'train', 'val']
train_data_dir = ['TRAIN_LGHG2@n10degC_to_25degC_Norm_5Inputs.csv']
val_data_dir = ['01_TEST_LGHG2@n10degC_Norm_(05_Inputs).csv', '02_TEST_LGHG2@0degC_Norm_(05_Inputs).csv', '03_TEST_LGHG2@10degC_Norm_(05_Inputs).csv', '04_TEST_LGHG2@25degC_Norm_(05_Inputs).csv']
test_data_dir = ['01_TEST_LGHG2@n10degC_Norm_(05_Inputs).csv', '02_TEST_LGHG2@0degC_Norm_(05_Inputs).csv', '03_TEST_LGHG2@10degC_Norm_(05_Inputs).csv', '04_TEST_LGHG2@25degC_Norm_(05_Inputs).csv']
models_dir = ['.ipynb_checkpoints', 'dl', 'ml']
dl_models_dir = ['rnn_model.keras']
ml_models_dir = ['soc_estimation_ml_decisiontree.sav', 'soc_estimation_ml_lregression.sav', 'soc_estimation_ml_nnregression.sav']


In [6]:
import pandas as pd
import numpy as np

## Training Data

In [7]:
train_data_filename = os.listdir(config.get_train_data_dir())[0]
train_data_path = os.path.join(config.get_train_data_dir(), train_data_filename)
train_data = pd.read_csv(train_data_path)
train_data.columns

Index(['V', 'I', 'Temp', 'V_avg', 'I_avg', 'SOC'], dtype='object')

In [8]:
cols = set(train_data.columns)
targets = ['SOC']
targets = set(targets)
features = cols - targets
targets = list(targets)
features = list(features)
print(f'features = {features}')
print(f'targets = {targets}')

targets_filename = 'targets.txt'
targets_path = os.path.join(config.get_data_dir(), targets_filename)
with open(targets_path, 'w') as file:
    for i in range(len(targets)):
        file.write(f'{targets[i]}\n')

features_filename = 'features.txt'
features_path = os.path.join(config.get_data_dir(), features_filename)
with open(features_path, 'w') as file:
    for i in range(len(features)):
        file.write(f'{features[i]}\n')

features = ['I_avg', 'V_avg', 'Temp', 'I', 'V']
targets = ['SOC']


### Extracting Features for Training

In [9]:
X_train = train_data[features].values
X_train.shape

(669956, 5)

In [10]:
X_train

array([[0.75102009, 0.38514793, 0.30310108, 0.75102009, 0.38514793],
       [0.75102009, 0.38514988, 0.30459129, 0.75102009, 0.38515183],
       [0.75102009, 0.38515183, 0.3060815 , 0.75102009, 0.38515573],
       ...,
       [0.75102009, 0.45983939, 0.00847709, 0.75102009, 0.47884278],
       [0.75102009, 0.45997861, 0.00847709, 0.75102009, 0.4789612 ],
       [0.75102009, 0.46011672, 0.00847709, 0.75102009, 0.4789612 ]])

### Extracting Target Variable for Training

In [11]:
y_train = train_data[targets].values
y_train.shape

(669956, 1)

In [12]:
y_train

array([[0.20641667],
       [0.20641667],
       [0.20641667],
       ...,
       [0.28324333],
       [0.28324333],
       [0.28324333]])

## Data Normalization

In [13]:
import utils

In [14]:
normalized_X_train = utils.normalize(X_train)
normalized_X_train

array([[ 1.        , -0.63365322, -1.        ,  1.        , -0.63365322],
       [ 1.        , -0.63909772, -1.        ,  1.        , -0.63908898],
       [ 1.        , -0.64457869, -1.        ,  1.        , -0.64456116],
       ...,
       [ 1.        ,  0.21572031, -1.        ,  1.        ,  0.26690492],
       [ 1.        ,  0.21609528, -1.        ,  1.        ,  0.26722387],
       [ 1.        ,  0.21646728, -1.        ,  1.        ,  0.26722387]])

# Linear Regression

In [15]:
from sklearn import linear_model

lregression_model = linear_model.LinearRegression()
lregression_model

LinearRegression()

In [16]:
lregression_model.fit(
    X=normalized_X_train,
    y=y_train
)

LinearRegression()

In [17]:
lregression_model_path = './models/ml/soc_estimation_ml_lregression.sav'
pickle.dump(lregression_model, open(lregression_model_path, 'wb'))

# Decision Tree

In [18]:
from sklearn import tree

decisiontree_model = tree.DecisionTreeRegressor()
decisiontree_model

DecisionTreeRegressor()

In [19]:
decisiontree_model.fit(
    X=normalized_X_train, 
    y=y_train
)

DecisionTreeRegressor()

In [20]:
decisiontree_model_path = './models/ml/soc_estimation_ml_decisiontree.sav'
pickle.dump(decisiontree_model, open(decisiontree_model_path, 'wb'))

# Nearest Neighbors Regression

In [21]:
from sklearn import neighbors

n_neighbors = 5

nnregression_model = neighbors.KNeighborsRegressor(n_neighbors)
nnregression_model

KNeighborsRegressor()

In [22]:
nnregression_model.fit(
    X=normalized_X_train, 
    y=y_train
)

KNeighborsRegressor()

In [23]:
nnregression_model_path = './models/ml/soc_estimation_ml_nnregression.sav'
pickle.dump(nnregression_model, open(nnregression_model_path, 'wb'))